# FlightLog - Elegant Integration Example

This notebook demonstrates clean, production-ready integration of FlightLog for real-time backtest monitoring.

## Setup Instructions

**Terminal 1: Start FlightLog**
```bash
docker compose exec zipline-jupyter bash
python scripts/flightlog.py --host 0.0.0.0 --level INFO
```

**Terminal 2: Run this notebook**

Watch Terminal 1 for real-time colored logs!

## Step 1: Import and Setup

In [ ]:
import logging
import pandas as pd
from zipline import run_algorithm
from zipline.api import (
    order_target_percent,
    symbol,
    schedule_function,
    date_rules,
    time_rules,
)
from zipline.utils.progress import enable_progress_logging
from zipline.utils.flightlog_client import enable_flightlog, log_to_flightlog

# Setup logging
logging.basicConfig(level=logging.INFO)

## Step 2: Connect to FlightLog

One line connects to the FlightLog server running in Terminal 1:

In [ ]:
# Connect to FlightLog (localhost when running inside Docker)
connected = enable_flightlog(host='localhost', port=9020)

if connected:
    print("✓ Connected to FlightLog!")
    print("  Watch Terminal 1 for real-time logs")
else:
    print("⚠ FlightLog not running - logs appear here only")

## Step 3: Enable Progress Logging

Show real-time progress bars with portfolio metrics:

In [ ]:
# Enable progress logging with QuantRocket-style output
enable_progress_logging(
    algo_name='My-Strategy',
    update_interval=5  # Log every 5 trading days
)

## Step 4: Define Your Strategy

Use `log_to_flightlog()` for key events you want to monitor:

In [ ]:
def initialize(context):
    """
    Initialize strategy - use FlightLog for important events.
    """
    # Log initialization
    log_to_flightlog('Strategy initializing...', level='INFO')
    
    # Setup portfolio
    context.stocks = [
        symbol('AAPL'),
        symbol('MSFT'),
        symbol('GOOGL'),
    ]
    
    # Schedule rebalancing
    schedule_function(
        rebalance,
        date_rules.week_start(),
        time_rules.market_open()
    )
    
    log_to_flightlog(
        f'Initialized with {len(context.stocks)} stocks',
        level='INFO'
    )


def rebalance(context, data):
    """
    Rebalance portfolio weekly - log key decisions.
    """
    # Equal weight allocation
    weight = 1.0 / len(context.stocks)
    
    # Place orders
    for stock in context.stocks:
        if data.can_trade(stock):
            order_target_percent(stock, weight)
    
    # Log rebalancing (only on actual rebalance days)
    portfolio_value = context.portfolio.portfolio_value
    positions = len(context.portfolio.positions)
    
    log_to_flightlog(
        f'Rebalanced: {positions} positions, Portfolio: ${portfolio_value:,.0f}',
        level='INFO'
    )


def analyze(context, perf):
    """
    Final analysis - log summary to FlightLog.
    """
    total_return = perf['algorithm_period_return'].iloc[-1] * 100
    final_value = perf['portfolio_value'].iloc[-1]
    
    log_to_flightlog('=' * 50, level='INFO')
    log_to_flightlog('BACKTEST COMPLETE', level='INFO')
    log_to_flightlog(f'Total Return: {total_return:+.2f}%', level='INFO')
    log_to_flightlog(f'Final Value: ${final_value:,.0f}', level='INFO')
    log_to_flightlog('=' * 50, level='INFO')

## Step 5: Run the Backtest

Watch Terminal 1 for real-time logs with colors!

In [ ]:
# Run backtest
result = run_algorithm(
    start=pd.Timestamp('2023-01-01'),
    end=pd.Timestamp('2023-12-31'),
    initialize=initialize,
    analyze=analyze,
    capital_base=100000,
    bundle='sharadar'
)

print("\n✓ Backtest complete!")
print(f"Total Return: {result['algorithm_period_return'].iloc[-1] * 100:+.2f}%")
print(f"Final Value: ${result['portfolio_value'].iloc[-1]:,.0f}")

## Step 6: Analyze Results

In [ ]:
# Plot results
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Portfolio value
result['portfolio_value'].plot(ax=ax1, title='Portfolio Value')
ax1.set_ylabel('Value ($)')
ax1.grid(True)

# Cumulative returns
(result['algorithm_period_return'] * 100).plot(
    ax=ax2,
    title='Cumulative Returns'
)
ax2.set_ylabel('Return (%)')
ax2.grid(True)

plt.tight_layout()
plt.show()

## Best Practices

### ✅ DO:
- Log **important events**: initialization, rebalancing, errors
- Use **appropriate log levels**: INFO for normal, WARNING for issues, ERROR for failures
- Log **summary metrics**: portfolio value, position counts, returns
- Keep log messages **concise and informative**

### ❌ DON'T:
- Log every `handle_data()` call (too noisy)
- Log verbose debug info in production
- Duplicate information already in progress bars
- Log sensitive information (API keys, etc.)

## Log Levels Guide

```python
# DEBUG: Detailed diagnostic info
log_to_flightlog('Calculating momentum for AAPL', level='DEBUG')

# INFO: General informational messages
log_to_flightlog('Rebalanced portfolio', level='INFO')

# WARNING: Warning messages
log_to_flightlog('Low liquidity for TSLA', level='WARNING')

# ERROR: Error messages
log_to_flightlog('Failed to place order', level='ERROR')

# CRITICAL: Critical issues
log_to_flightlog('Risk limit breached!', level='CRITICAL')
```

## Tips

1. **Start FlightLog first**, then run your backtest
2. **Use localhost** when running inside Docker
3. **Set update_interval** based on backtest length (1-5 for short, 20+ for long)
4. **Check connection** before running long backtests
5. **Save FlightLog output** with `--file` flag for later review

## Summary

FlightLog gives you:
- ✅ Real-time monitoring in separate terminal
- ✅ Color-coded log levels
- ✅ Progress bars with live metrics
- ✅ Clean separation of logs from notebook output
- ✅ Persistent log files for review